In [4]:
import typing

import numpy as np
import pandas as pd

from src import load_data
from src import embeddings

df_b, df_r = load_data.load_yelp_data()
emb_r = np.lib.format.open_memmap("data/r_emb_large_03.npy")

In [8]:
e1 = embeddings.embedding("affordable")
df_r["d1"] = embeddings.cos_sim(e1, emb_r, batch_size=20000)
df_r.sort_values("d1", ascending=False).head(20)[["stars", "text", "d1"]]

100%|██████████| 49/49 [00:10<00:00,  4.52it/s]


,stars,text,d1
717633,4,Delicious..yet affordable!,0.517306
318632,5,nice and not expensive. Especially for Chinese...,0.473827
817970,5,"not fancy, a little messy, but trust-worthy an...",0.453506
351114,5,Reasonable price and decent food. Very accessi...,0.450593
84573,4,Clean!!! An affordable with plenty of options ...,0.436240
28463,4,consistently good and not overpriced,0.433797
461993,5,"Great, affordable and effective acupuncture! I...",0.424710
544869,3,"Its good but not great. affordable, fast. so s...",0.420634
123638,3,Acceptable service and prices.,0.417170
381505,5,"Affordable, good quality and service, perfect ...",0.415132
